In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/cleaned-data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleaned-data/Cleaned_medquad.csv
/kaggle/input/cleaned-data/cleaned_exercises.csv
/kaggle/input/cleaned-data/cleaned_supplements_instruction_response.csv
/kaggle/input/cleaned-data/cleaned_program_summary.csv
/kaggle/input/cleaned-data/cleaned_calories_burn.csv
/kaggle/input/cleaned-data/cleaned_fitness_exercise.csv
/kaggle/input/cleaned-data/cleaned_food_nutrition_with_categories.csv
/kaggle/input/cleaned-data/cleaned_gym_members_tracking.csv
/kaggle/input/cleaned-data/cleaned_FITNESS_DATASET.csv
/kaggle/input/cleaned-data/cleaned_programs_detailed.csv
/kaggle/input/cleaned-data/cleaned_json_exercises.csv
/kaggle/input/cleaned-data/cleaned_megaGymDataset.csv
/kaggle/input/cleaned-data/cleaned_recipies_detailed.csv
/kaggle/input/cleaned-data/cleaned_conversational_dataset.csv
/kaggle/input/cleaned-data/cleaned_supplements_master.csv


In [2]:
# ==============================================================================
# FINAL KAGGLE FINE-TUNING SCRIPT (Version 14 - With Clean Data)
# This is the definitive script for fine-tuning on your clean dataset.
# ==============================================================================

# --- STAGE 1: INSTALL UNSLOTH ---
# Unsloth handles the installation of its own optimized libraries.
print("--- STAGE 1: INSTALLING UNSLOTH ---")
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"
print("--- INSTALLATION COMPLETE ---")


# --- STAGE 2: LOGIN AND IMPORTS ---
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
import os
import shutil # For zipping the output

print("\n--- STAGE 2: AUTHENTICATING AND IMPORTING ---")
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
login(token=hf_token)
print("--- AUTHENTICATION COMPLETE ---")


# --- STAGE 3: CONFIGURATION ---
print("\n--- STAGE 3: CONFIGURING PATHS AND PARAMETERS ---")
BASE_MODEL_ID = "google/gemma-2b-it"
# Kaggle creates a directory from your zip file name.
# IMPORTANT: It converts underscores and spaces to hyphens.
DATA_FOLDER = "/kaggle/input/cleaned-data"
NEW_ADAPTER_NAME = "gemma_gym_coach_v2" # Version 2, trained on clean data
print("--- CONFIGURATION COMPLETE ---")


# --- STAGE 4: DATASET PREPARATION ---
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    responses = examples["response"]
    texts = []
    for instruction, response in zip(instructions, responses):
        text = f"<start_of_turn>user\n{instruction}<end_of_turn>\n<start_of_turn>model\n{response}<end_of_turn>"
        texts.append(text)
    return { "text" : texts, }

print("\n--- STAGE 4: PREPARING DATASET ---")
all_csv_files = os.path.join(DATA_FOLDER, "*.csv")
dataset = load_dataset("csv", data_files=all_csv_files, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)
print("--- DATASET READY ---")


# --- STAGE 5: LOAD THE MODEL USING UNSLOTH ---
max_seq_length = 2048
print(f"\n--- STAGE 5: LOADING MODEL '{BASE_MODEL_ID}' VIA UNSLOTH ---")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL_ID,
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)
print("--- MODEL LOADED ---")


# --- STAGE 6: CONFIGURE PEFT (LoRA) ---
print("\n--- STAGE 6: CONFIGURING LoRA ADAPTER ---")
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length,
)
model.print_trainable_parameters()
print("--- LoRA CONFIGURED ---")


# --- STAGE 7: TRAINER SETUP ---
print("\n--- STAGE 7: SETTING UP THE TRAINER ---")
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 1000,         # Train for longer
        learning_rate = 2e-5,     # Train much more gently
        # --------------------------------
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)
print("--- TRAINER READY ---")


# --- STAGE 8: START TRAINING ---
print("\n--- STAGE 8: STARTING TRAINING ---")
trainer_stats = trainer.train()
print("--- TRAINING FINISHED ---")


# --- STAGE 9: SAVE AND ZIP THE FINAL ADAPTER ---
print(f"\n--- STAGE 9: SAVING AND ZIPPING THE ADAPTER ---")
model.save_pretrained(NEW_ADAPTER_NAME)

output_folder_path = f"/kaggle/working/{NEW_ADAPTER_NAME}"
zip_output_path = f"/kaggle/working/{NEW_ADAPTER_NAME}" # .zip is added by make_archive
shutil.make_archive(
    base_name=zip_output_path,
    format='zip',
    root_dir=output_folder_path
)

print(f"\n--- SUCCESS! ---")
print(f"--- Adapter saved and zipped to: {zip_output_path}.zip ---")
print("--- You can now download this file and use it in your inference notebook. ---")

--- STAGE 1: INSTALLING UNSLOTH ---
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-vlcx4ffw/unsloth_6d3b05b4c91a4a51a38fd6012dfb225b
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-vlcx4ffw/unsloth_6d3b05b4c91a4a51a38fd6012dfb225b
  Resolved https://github.com/unslothai/unsloth.git to commit b3e81e2556e9de09dcaec964b48193940f7592e0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 29.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.4 MB/s eta 0:00:00:00:0100:01
 

2025-10-26 13:19:37.948606: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761484778.340938      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761484778.455179      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!

--- STAGE 2: AUTHENTICATING AND IMPORTING ---
--- AUTHENTICATION COMPLETE ---

--- STAGE 3: CONFIGURING PATHS AND PARAMETERS ---
--- CONFIGURATION COMPLETE ---

--- STAGE 4: PREPARING DATASET ---


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/88708 [00:00<?, ? examples/s]

...

--- TRAINER READY ---

--- STAGE 8: STARTING TRAINING ---


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 88,708 | Num Epochs = 1 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 19,611,648 of 2,525,784,064 (0.78% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,10.955900
20,8.727600
30,6.489100
40,3.548700
50,2.975400
60,3.031000
70,2.918800
80,2.474600
90,2.109300
100,2.165300


--- TRAINING FINISHED ---

--- STAGE 9: SAVING AND ZIPPING THE ADAPTER ---

--- SUCCESS! ---
--- Adapter saved and zipped to: /kaggle/working/gemma_gym_coach_v2.zip ---
--- You can now download this file and use it in your inference notebook. ---


In [4]:
# ==============================================================================
# Cell 2: Zip the Output for Download
# ==============================================================================
import os
import shutil # <-- The missing import statement

# --- Configuration ---
# This must match the name you used in the training script
ADAPTER_FOLDER_NAME = "gemma_gym_coach_v2"

# --- Zipping Logic ---
# Define the full path to the folder you want to zip
folder_to_zip = f"/kaggle/working/{ADAPTER_FOLDER_NAME}"

# Define the name of the output zip file
zip_file_name = f"/kaggle/working/{ADAPTER_FOLDER_NAME}" # .zip is added automatically

if os.path.exists(folder_to_zip):
    print(f"--- Found adapter folder: '{folder_to_zip}' ---")
    print(f"--- Creating zip file for easy download... ---")

    # Use the shutil library to create the zip archive
    shutil.make_archive(
        base_name=zip_file_name, # The name of the file to create (e.g., /kaggle/working/gemma_gym_coach_v1_unsloth)
        format='zip',           # The archive format
        root_dir=folder_to_zip  # The directory to zip
    )
    
    print(f"\nSUCCESS: Created zip file at '{zip_file_name}.zip'")
    print("You can now find this file in the '/kaggle/working/' directory and download it.")
else:
    print(f"ERROR: Could not find the directory '{folder_to_zip}'. Please make sure the training script finished successfully and the adapter name is correct.")

--- Found adapter folder: '/kaggle/working/gemma_gym_coach_v2' ---
--- Creating zip file for easy download... ---

SUCCESS: Created zip file at '/kaggle/working/gemma_gym_coach_v2.zip'
You can now find this file in the '/kaggle/working/' directory and download it.
